In [1]:
from IPython.core.display import display, HTML
from IPython.display import Image, display
from IPython.utils.text import columnize
#display(HTML("<style>.container { width:100% !important; }</style>"))
import h3

import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

import statistics
import statsmodels as sm
import statsmodels.formula.api as sm_formula
from scipy import stats

import warnings
warnings.filterwarnings('ignore')


# don't use scientific notation
np.set_printoptions(suppress=True) 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import h3

import geopandas as gpd

from shapely import geometry, ops
import libpysal as pys
import esda
import pointpats as pp

from geojson.feature import *

from annoy import AnnoyIndex

import bisect
import itertools
from more_itertools import unique_everseen

import math
import random
import decimal
from collections import Counter

from pprint import pprint
import copy

from tqdm import tqdm

import pydeck

from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear
import folium

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.gridspec as gridspec

from PIL import Image as pilim

%matplotlib inline

import sys
sys.path.append('./PlotNeuralNet')

from PlotNeuralNet.pycore.tikzeng import *
from PlotNeuralNet.pycore.blocks import *

%load_ext line_profiler
%load_ext pycodestyle_magic

/tmp/ipykernel_21934/2588269692.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def load_and_prepare_busstops(filepath):
    """Loads a geojson files of point geometries and features,
    extracts the latitude and longitude into separate columns,
    deduplicates busstops (since multiple buslines share them)"""

    gdf_raw = gpd.read_file(filepath, driver="GeoJSON")
    print("Total number of bus stops in original dataset", gdf_raw.shape[0]) 

    gdf_raw["latitude"] = gdf_raw["geometry"].apply(lambda p: p.y)
    gdf_raw["longitude"] = gdf_raw["geometry"].apply(lambda p: p.x)

    # reset index to store it in a column
    gdf_raw.reset_index(inplace=True, drop = False)
    
    return gdf_raw

In [101]:
gdf_raw = gpd.read_file("datasets_demo/imoveis.geojson", driver="GeoJSON")
gdf_raw["geometry"].apply(lambda p: p.outer)

In [213]:
DADOS = "APS"

In [214]:
if DADOS == "TAXI":
    input_file_busstops = "datasets_demo/taxis.geojson"
elif DADOS == "STCP":
    input_file_busstops = "datasets_demo/stcp-stops.geojson"
elif DADOS == "SCOOTER_START":
    input_file_busstops = "datasets_demo/trip_starts.json"    
elif DADOS == "SCOOTER_END":
    input_file_busstops = "datasets_demo/trip_ends_2.json"   
elif DADOS == "CINEMAS":
    input_file_busstops = "datasets_demo/cinemas.geojson"       
elif DADOS == "ARTE":
    input_file_busstops = "datasets_demo/estatuas.geojson"
elif DADOS == "IMOVEIS_VALOR":
    input_file_busstops = "datasets_demo/imoveis.geojson"
elif DADOS == "BIBLIO":
    input_file_busstops = "datasets_demo/pontos-de-interesse-bibliotecas-arquivos-e-centros-de-documentacao.json"    
elif DADOS == "EXPOSICOES":
    input_file_busstops = "datasets_demo/pontos-de-interesse-centros-de-exposicoes-galerias-de-arte.json"    
elif DADOS == "MIRADOUROS":
    input_file_busstops = "datasets_demo/pontos-de-interesse-cultura-e-patrimonio-miradouros.json"    
elif DADOS == "MONUMENTOS":
    input_file_busstops = "datasets_demo/pontos-de-interesse-cultura-e-patrimonio-monumentos.json"    
elif DADOS == "MUSEUS":
    input_file_busstops = "datasets_demo/pontos-de-interesse-cultura-e-patrimonio-museus-e-centros-tematicos.json"
elif DADOS == "CONCERTOS":
    input_file_busstops = "datasets_demo/pontos-de-interesse-cultura-e-patrimonio-salas-de-concerto.json"
elif DADOS == "TEATROS":
    input_file_busstops = "datasets_demo/pontos-de-interesse-cultura-e-patrimonio-teatros.json"
elif DADOS == "APS":
    input_file_busstops = "datasets_demo/wifi-aps.json"    


    
gdf_raw = load_and_prepare_busstops(filepath = input_file_busstops)

if DADOS == "STCP":
    gdf_raw = gdf_raw.drop_duplicates('stop_id')
    print("after cleaning: " + str(gdf_raw.shape[0]))

# display first 5 rows of the geodataframe, transposed
gdf_raw.head().T

Total number of bus stops in original dataset 122142


,0,1,2,3,4
index,0,1,2,3,4
Access_Point,a0-23-9f-ca-a4-60,60-de-f3-96-10-60,a0-23-9f-ca-86-60,40-01-7a-53-23-40,a0-23-9f-9e-a5-c0
Date_day,2022-03-30,2021-05-12,2021-07-28,2021-08-16,2022-03-14
geometry,POINT (-8.601628 41.146193),POINT (-8.601628 41.146193),POINT (-8.601628 41.146193),POINT (-8.601628 41.146193),POINT (-8.601628 41.146193)
latitude,41.14619,41.14619,41.14619,41.14619,41.14619
longitude,-8.60163,-8.60163,-8.60163,-8.60163,-8.60163


In [226]:
gdf_raw["geometry"].value_counts()

POINT (-8.60163 41.14619)    122142
Name: geometry, dtype: int64

In [215]:
#ite = gdf_raw.sample(n=5_000, random_state=42)
#gdf_raw = ite

In [216]:
gdf_raw_cpy = gdf_raw.reset_index(inplace = False, drop = False)
df_stops_to_buslines = gdf_raw_cpy

In [217]:
# index each data point into the spatial index of the specified resolution
for res in range(7, 11):
    col_hex_id = "hex_id_{}".format(res)
    col_geom = "geometry_{}".format(res)
    msg_ = "At resolution {} -->  H3 cell id : {} and its geometry: {} "
    print(msg_.format(res, col_hex_id, col_geom))

    df_stops_to_buslines[col_hex_id] = df_stops_to_buslines.apply(
                                        lambda row: h3.geo_to_h3(
                                                    lat = row["latitude"],
                                                    lng = row["longitude"],
                                                    resolution = res),
                                        axis = 1)

    # use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    df_stops_to_buslines[col_geom] = df_stops_to_buslines[col_hex_id].apply(
                                        lambda x: {"type": "Polygon",
                                                   "coordinates":
                                                   [h3.h3_to_geo_boundary(
                                                       h=x, geo_json=True)]
                                                   }
                                         )
# transpose for better display
df_stops_to_buslines.head().T

At resolution 7 -->  H3 cell id : hex_id_7 and its geometry: geometry_7 
At resolution 8 -->  H3 cell id : hex_id_8 and its geometry: geometry_8 
At resolution 9 -->  H3 cell id : hex_id_9 and its geometry: geometry_9 
At resolution 10 -->  H3 cell id : hex_id_10 and its geometry: geometry_10 


,0,1,2,3,4
level_0,0,1,2,3,4
index,0,1,2,3,4
Access_Point,a0-23-9f-ca-a4-60,60-de-f3-96-10-60,a0-23-9f-ca-86-60,40-01-7a-53-23-40,a0-23-9f-9e-a5-c0
Date_day,2022-03-30,2021-05-12,2021-07-28,2021-08-16,2022-03-14
geometry,POINT (-8.601628 41.146193),POINT (-8.601628 41.146193),POINT (-8.601628 41.146193),POINT (-8.601628 41.146193),POINT (-8.601628 41.146193)
latitude,41.14619,41.14619,41.14619,41.14619,41.14619
longitude,-8.60163,-8.60163,-8.60163,-8.60163,-8.60163
hex_id_7,8739220f1ffffff,8739220f1ffffff,8739220f1ffffff,8739220f1ffffff,8739220f1ffffff
geometry_7,"{'type': 'Polygon', 'coordinates': [((-8.59958...","{'type': 'Polygon', 'coordinates': [((-8.59958...","{'type': 'Polygon', 'coordinates': [((-8.59958...","{'type': 'Polygon', 'coordinates': [((-8.59958...","{'type': 'Polygon', 'coordinates': [((-8.59958..."
hex_id_8,8839220f15fffff,8839220f15fffff,8839220f15fffff,8839220f15fffff,8839220f15fffff


In [220]:
def counts_by_hexagon(df, res):
    """Aggregates the number of busstops at hexagon level"""

    col_hex_id = "hex_id_{}".format(res)
    col_geometry = "geometry_{}".format(res)

    # within each group preserve the first geometry and count the ids
    df_aggreg = df.groupby(by = col_hex_id).agg({col_geometry: "first",
                                                "latitude": "count"})

    df_aggreg.reset_index(inplace = True)
    df_aggreg.rename(columns={"latitude": "value"}, inplace = True)

    df_aggreg.sort_values(by = "value", ascending = False, inplace = True)
    return df_aggreg

In [221]:
# demo at resolution 8
RES = 9
df_aggreg_8 = counts_by_hexagon(df = df_stops_to_buslines, res = RES)
print(df_aggreg_8.shape)
df_aggreg_8.head(5)

(1, 3)


,hex_id_9,geometry_9,value
0,8939220f143ffff,"{'type': 'Polygon', 'coordinates': [((-8.60229...",122142


In [222]:
def base_empty_map():
    """Prepares a folium map centered in a central GPS point of Toulouse"""
    m = Map(location =  [41.14961,-8.61099],
            zoom_start = 9.5,
            tiles = "cartodbpositron",
            attr = '''© <a href="http://www.openstreetmap.org/copyright">
                      OpenStreetMap</a>contributors ©
                      <a href="http://cartodb.com/attributions#basemaps">
                      CartoDB</a>'''
            )
    return m

def hexagons_dataframe_to_geojson(df_hex, hex_id_field,
                                  geometry_field, value_field,
                                  file_output = None):

    """Produce the GeoJSON representation containing all geometries in a dataframe
     based on a column in geojson format (geometry_field)"""

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry = row[geometry_field],
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    geojson_result = json.dumps(feat_collection)

    # optionally write to file
    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    return geojson_result


# --------------------------------------------------------------------


def choropleth_map(df_aggreg, hex_id_field, geometry_field, value_field,
                   layer_name, initial_map = None, kind = "linear",
                   border_color = 'black', fill_opacity = 0.7,
                   with_legend = False):

    """Plots a choropleth map with folium"""

    if initial_map is None:
        initial_map = base_empty_map()

    # the custom colormap depends on the map kind
    if kind == "linear":
        min_value = df_aggreg[value_field].min()
        max_value = df_aggreg[value_field].max()
        m = round((min_value + max_value) / 2, 0)
        custom_cm = cm.LinearColormap(['green', 'yellow', 'red'],
                                      vmin = min_value,
                                      vmax = max_value)
    elif kind == "outlier":
        # for outliers, values would be -1,0,1
        custom_cm = cm.LinearColormap(['blue', 'white', 'red'],
                                      vmin=-1, vmax=1)
    elif kind == "filled_nulls":
        min_value = df_aggreg[df_aggreg[value_field] > 0][value_field].min()
        max_value = df_aggreg[df_aggreg[value_field] > 0][value_field].max()
        m = round((min_value + max_value) / 2, 0)
        custom_cm = cm.LinearColormap(['silver', 'green', 'yellow', 'red'],
                                      index = [0, min_value, m, max_value],
                                      vmin = min_value,
                                      vmax = max_value)

    # create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_aggreg, hex_id_field,
                                                 geometry_field, value_field)

    # plot on map
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity
        },
        name = layer_name
    ).add_to(initial_map)

    # add legend (not recommended if multiple layers)
    if with_legend is True:
        custom_cm.add_to(initial_map)

    return initial_map

In [223]:
m_hex = choropleth_map(df_aggreg = df_aggreg_8,
                       hex_id_field = "hex_id_"+str(RES),
                       geometry_field = f"geometry_"+str(RES),
                       value_field = "value",
                       layer_name = "Choropleth 8",
                       with_legend = True)
m_hex

In [212]:
df_aggreg_8.to_csv("./dados_finais/" + DADOS + ".csv")

## Map point counts

In [25]:
# quick visualization on map of raw data

m = base_empty_map()
mc = MarkerCluster()

gdf_dedup = gdf_raw.drop_duplicates(subset=["latitude", "longitude"])
print("Total number of bus stops in deduplicated dataset", gdf_dedup.shape[0]) 

for i, row in gdf_dedup.iterrows():
    mk = Marker(location=[row["latitude"], row["longitude"]])
    mk.add_to(mc)

mc.add_to(m)
m

Total number of bus stops in deduplicated dataset 2490


In [227]:
import os

In [229]:
os.listdir("./dados_finais")

['arte_res9.csv',
 'biblio_res9.csv',
 'cinemas_res9.csv',
 'MUSEUS.csv',
 'MONUMENTOS.csv',
 'trip_ends_res9.csv',
 'bus_stops_res_9.csv',
 'estatuas_res9.csv',
 'MIRADOUROS.csv',
 'CONCERTOS.csv',
 'SCOOTER_END.csv',
 'taxis_res9.csv',
 'exposicoes.csv',
 'TEATROS.csv',
 'trips_starts_res9.csv']